<a href="https://colab.research.google.com/github/MuhammadHelmyOmar/ArabicPIIRedaction/blob/main/fine-tuning_SLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [52]:
import pandas as pd

### Test data
- Original Competition Data

In [ ]:
data_url = "https://raw.githubusercontent.com/MuhammadHelmyOmar/ArabicPIIRedaction/main/data/data_final.xlsx"

test = pd.read_excel(data_url)

test.head()

,source,target,dialect
0,الأستاذ أدولفوس ريغان زييمان، بصفتك المدير الت...,الأستاذ [MASK]، بصفتك المدير التنفيذي الرئيسي ...,EGYPT
1,مرحبًا يا هانا، ممكن لو سمحتي تتحققي من التداع...,مرحبًا يا [MASK]، ممكن لو سمحتي تتحققي من التد...,EGYPT
2,نطلب أيضًا مراجعة سياساتنا فيما يتعلق بالتغيير...,نطلب أيضًا مراجعة سياساتنا فيما يتعلق بالتغيير...,SAUDI
3,عزيزي ديفان، مطلوب تقديم عرض تقديمي على مستوى ...,عزيزي [MASK]، مطلوب تقديم عرض تقديمي على مستوى...,EGYPT
4,ممكن كمان نعمل جلسة عن كيفية إدارة التوتر والت...,The sentence doesn't contain any personal or s...,LEVANTINE


### Train data
- Created by Mosaab

In [ ]:
from google.colab import drive
import random

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = "/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/ar_dataset_translated_fixed.csv"

train = pd.read_csv(data_path)

In [ ]:
train.drop(["Unnamed: 0",'lang'], axis=1, inplace=True)
print(len(train))
train.head()

209261


,source,tokens,tags,clean_source
0,مرحبًا <!FIRSTNAME_1!> Johan </!FIRSTNAME_1!> ...,"['مرحبا', 'Johan', 'Walker', '،', 'إشعار', 'ال...","['O', 'B-FIRSTNAME', 'B-LASTNAME', 'O', 'O', '...",مرحبا Johan Walker، إشعار الامتثال: تتطلب التغ...
1,نحن نقوم بمراجعة القضية التي رفعها <!FIRSTNAME...,"['نحن', 'نقوم', 'بمراجعة', 'القضية', 'التي', '...","['O', 'O', 'O', 'O', 'O', 'O', 'B-FIRSTNAME', ...",نحن نقوم بمراجعة القضية التي رفعها كادي شوبي ب...
2,بسبب اتفاقية السرية، لا يمكن للمريضة <!SEX_1!>...,"['بسبب', 'اتفاقية', 'السرية', '،', 'لا', 'يمكن...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEX', '...",بسبب اتفاقية السرية، لا يمكن للمريضة أنثى الدر...
3,عزيزي ولي الأمر، يلزم الحصول على إذن في <!DATE...,"['عزيزي', 'ولي', 'الأمر', '،', 'يلزم', 'الحصول...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",عزيزي ولي الأمر، يلزم الحصول على إذن في 2018-0...
4,تنظيم اجتماع حول <!AGE_1!> 7 سنوات </!AGE_1!> ...,"['تنظيم', 'اجتماع', 'حول', '7', 'سنوات', '(', ...","['O', 'O', 'O', 'B-AGE', 'I-AGE', 'O', 'B-GEND...",تنظيم اجتماع حول 7 سنوات (Genderqueer) وسلامة ...


In [ ]:
idx = random.randint(0, len(train))
idx

33257

In [ ]:
print(train.tokens[idx])
print(train.tags[idx])
print(len(train.tokens[idx]))
print(len(train.tags[idx]))

['مرحبا', 'Reed', '،', 'لقد', 'واجهنا', 'مشكلة', 'في', 'معالجة', 'المدفوعات', 'الدولية', 'لمزود', 'تكنولوجيا', 'التعليم', 'الجديد', 'لدينا', '.', 'هل', 'يمكنك', 'التحقق', 'من', 'المشكلة', 'مع', 'BIC', 'LERQNZJBXXX', 'و', 'IBAN', 'NO2484520800325', '؟']
['O', 'B-FIRSTNAME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-BIC', 'O', 'O', 'B-IBAN', 'O']
252
159


##### Cleaning and preprocessing

In [ ]:
import re
import ast

In [ ]:
tokens_list = ast.literal_eval(train.tokens[idx])
print(tokens_list)
print(type(tokens_list), len(tokens_list))

tags_list = ast.literal_eval(train.tags[idx])
print(tags_list)
print(type(tags_list), len(tags_list))

['مرحبا', 'Reed', '،', 'لقد', 'واجهنا', 'مشكلة', 'في', 'معالجة', 'المدفوعات', 'الدولية', 'لمزود', 'تكنولوجيا', 'التعليم', 'الجديد', 'لدينا', '.', 'هل', 'يمكنك', 'التحقق', 'من', 'المشكلة', 'مع', 'BIC', 'LERQNZJBXXX', 'و', 'IBAN', 'NO2484520800325', '؟']
<class 'list'> 28
['O', 'B-FIRSTNAME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-BIC', 'O', 'O', 'B-IBAN', 'O']
<class 'list'> 28


In [ ]:
# Convert tokens and tags columns to lists

train['tokens'] = train["tokens"].apply(ast.literal_eval)
train['tags'] = train['tags'].apply(ast.literal_eval)

In [ ]:
train.head()

,source,tokens,tags,clean_source
0,مرحبًا <!FIRSTNAME_1!> Johan </!FIRSTNAME_1!> ...,"[مرحبا, Johan, Walker, ،, إشعار, الامتثال, :, ...","[O, B-FIRSTNAME, B-LASTNAME, O, O, O, O, O, O,...",مرحبا Johan Walker، إشعار الامتثال: تتطلب التغ...
1,نحن نقوم بمراجعة القضية التي رفعها <!FIRSTNAME...,"[نحن, نقوم, بمراجعة, القضية, التي, رفعها, كادي...","[O, O, O, O, O, O, B-FIRSTNAME, B-LASTNAME, O,...",نحن نقوم بمراجعة القضية التي رفعها كادي شوبي ب...
2,بسبب اتفاقية السرية، لا يمكن للمريضة <!SEX_1!>...,"[بسبب, اتفاقية, السرية, ،, لا, يمكن, للمريضة, ...","[O, O, O, O, O, O, O, B-SEX, O, O, O, B-PREFIX...",بسبب اتفاقية السرية، لا يمكن للمريضة أنثى الدر...
3,عزيزي ولي الأمر، يلزم الحصول على إذن في <!DATE...,"[عزيزي, ولي, الأمر, ،, يلزم, الحصول, على, إذن,...","[O, O, O, O, O, O, O, O, O, B-DATE, I-DATE, I-...",عزيزي ولي الأمر، يلزم الحصول على إذن في 2018-0...
4,تنظيم اجتماع حول <!AGE_1!> 7 سنوات </!AGE_1!> ...,"[تنظيم, اجتماع, حول, 7, سنوات, (, Genderqueer,...","[O, O, O, B-AGE, I-AGE, O, B-GENDER, O, O, O, ...",تنظيم اجتماع حول 7 سنوات (Genderqueer) وسلامة ...


In [ ]:
length_comparison_result = train.apply(lambda row: len(train.tokens) == len(train.tags), axis=1)
print(f"\nNumber of rows with mismatching lengths: {sum(~length_comparison_result)}")


Number of rows with mismatching lengths: 0


In [ ]:
tags_to_mask =  [
'EMAIL',
'IBAN',
'PERSONNAME',
'IPV6',
'MASKEDNUMBER',
'CREDITCARDNUMBER',
'LITECOINADDRESS',
'BIC',
'PHONENUMBER',
'ORDINALDIRECTION',
'VEHICLEVRM',
'CREDITCARDINUMBER',
'USERNAME',
'IPV4',
'PASSWORD',
'MAC',
'BITCOINADDRESS',
'SECONDARYADDRESS',
'CREDITCARDCVV',
'PHONEIMEI',
'ACCOUNTNAME',
'BUILDINGNUMBER',
'VEHICLEVIN',
'STREET',
'URL',
'IP',
'ZIPCODE',
'ACCOUNTNUMBER',
'FIRSTNAME',
'LASTNAME',
'MIDDLENAME',
'NEARBYGPSCOORDINATE',
'ETHEREUMADDRESS',
'PIN',
'SSN',
'COMPANYNAME',
'HEIGHT',
'CITY']

In [ ]:
def masker(row, tags_to_mask):
    """
    Constructs a masked sentence by iterating through tokens and replacing PII based on tags, attempting to preserve original formatting.

    Args:
    row: A pandas DataFrame row with 'tokens', 'tags', and 'clean_source' columns.
    tags_to_mask: List of tag entity types to mask.

    Returns:
    The constructed sentence with masked tokens for the given row, attempting to preserve spacing.
    """
    tokens = row['tokens']
    tags = row['tags']
    clean_source = row['clean_source']
    masked_sentence = ""
    current_position = 0

    for i in range(len(tokens)):
        token = tokens[i]
        tag = tags[i]

        # Find the position of the current token in the original string starting from the last processed position
        start_index = clean_source.find(token, current_position)

        if start_index != -1:
            # Append the text from the current position up to the start of the token
            masked_sentence += clean_source[current_position:start_index]

            if tag[2:] in tags_to_mask:
                masked_sentence += '[MASK]'
            else:
                masked_sentence += token

            # Update the current position to the end of the current token
            current_position = start_index + len(token)
        else:
            # If token not found at or after current_position, it indicates a discrepancy
            print(f"Warning: Token '{token}' not found in clean_source at or after position {current_position}")

    # Append any remaining text after the last token
    masked_sentence += clean_source[current_position:]

    return masked_sentence

In [ ]:
# Creating a new masked sentence

train['masked_source'] = train.apply(lambda row: masker(row, tags_to_mask), axis=1)

In [ ]:
train[['clean_source','masked_source']]

,clean_source,masked_source
0,مرحبا Johan Walker، إشعار الامتثال: تتطلب التغ...,مرحبا [MASK] [MASK]، إشعار الامتثال: تتطلب الت...
1,نحن نقوم بمراجعة القضية التي رفعها كادي شوبي ب...,نحن نقوم بمراجعة القضية التي رفعها [MASK] [MAS...
2,بسبب اتفاقية السرية، لا يمكن للمريضة أنثى الدر...,بسبب اتفاقية السرية، لا يمكن للمريضة أنثى الدر...
3,عزيزي ولي الأمر، يلزم الحصول على إذن في 2018-0...,عزيزي ولي الأمر، يلزم الحصول على إذن في 2018-0...
4,تنظيم اجتماع حول 7 سنوات (Genderqueer) وسلامة ...,تنظيم اجتماع حول 7 سنوات (Genderqueer) وسلامة ...
...,...,...
209256,عزيزتي الأنثى، حدث التعليم الفني في الشقة. تم ...,عزيزتي الأنثى، حدث التعليم الفني في [MASK][MAS...
209257,هل يمكننا ترتيب موعد مع والدي نيكو؟ لقد طلبت إ...,هل يمكننا ترتيب موعد مع والدي [MASK]؟ لقد طلبت...
209258,يرجى تأكيد استلام الزي الجديد عن طريق البريد ا...,يرجى تأكيد استلام الزي الجديد عن طريق البريد ا...
209259,أظهر الشخص ذو العيون ذات اللون البني وطوله 3' ...,أظهر الشخص ذو العيون ذات اللون البني وطوله [MA...


No charts were generated by quickchart


In [ ]:
train['tokens'][20]

['ترقيات',
 'المختبر',
 'في',
 'مدرستنا',
 'في',
 'إيست',
 'بيرسي',
 '.',
 'هل',
 'يمكننا',
 'الحصول',
 'على',
 'عنوان',
 'IP',
 '141',
 '.',
 '247',
 '.',
 '15',
 '.',
 '141',
 'وعنوان',
 'MAC',
 '8f',
 ':',
 '2f',
 ':',
 'd6',
 ':',
 '6f',
 ':',
 'f3',
 ':',
 '31',
 'للتكوين',
 '؟']

In [ ]:
train['tags'][20]

['O',
 'O',
 'O',
 'O',
 'O',
 'B-CITY',
 'I-CITY',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-IP',
 'I-IP',
 'I-IP',
 'I-IP',
 'I-IP',
 'I-IP',
 'I-IP',
 'O',
 'O',
 'B-MAC',
 'I-MAC',
 'I-MAC',
 'I-MAC',
 'I-MAC',
 'I-MAC',
 'I-MAC',
 'I-MAC',
 'I-MAC',
 'I-MAC',
 'I-MAC',
 'O',
 'O']

In [ ]:
train.clean_source[20]

'ترقيات المختبر في مدرستنا في إيست بيرسي. هل يمكننا الحصول على عنوان IP 141.247.15.141 وعنوان MAC 8f:2f:d6:6f:f3:31 للتكوين؟'

In [ ]:
masker(train.iloc[20], tags_to_mask)

'ترقيات المختبر في مدرستنا في [MASK] [MASK]. هل يمكننا الحصول على عنوان IP [MASK][MASK][MASK][MASK][MASK][MASK][MASK] وعنوان MAC [MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK] للتكوين؟'

In [ ]:
train.rename(columns={'clean_source': 'source', 'masked_source': 'target'}, inplace = True)

In [ ]:
# Save the train data after the updates
train[['source', 'target']].to_csv('/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/masked_train_data.csv', index=False)

##### Augmenting data with Egyptian dialect

In [51]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm.auto import tqdm
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 32.9 MB/s eta 0:00:00


In [53]:
# Config
input_csv = r"C:\Users\mosaabsayed\Downloads\ar_dataset_translated_fixed_cleaned.csv"
output_csv = r"C:\Users\mosaabsayed\Downloads\saudi_translated_sentences.csv"
target_dialect = "arabic-egyptian"  # choose from ['arabic-gulf', 'arabic-egyptian', 'arabic-levantine']
batch_size = 20_000
save_every = 10

# Chrome setup
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

service = Service()
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.lingobright.com/tools/arabic-dialect-translator/")

wait = WebDriverWait(driver, timeout=10, poll_frequency=0.2)

# Set dialects
wait.until(EC.presence_of_element_located((By.ID, "sourceDialect")))
driver.find_element(By.ID, "sourceDialect").find_element(By.XPATH, ".//option[@value='arabic-modern-standard']").click()
driver.find_element(By.ID, "targetDialect").find_element(By.XPATH, f".//option[@value='{target_dialect}']").click()

# Input/output areas and button
text_areas = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "text-area")))
input_box, output_box = text_areas[0], text_areas[1]
translate_button = driver.find_element(By.ID, "submitBtn")

# Load data
df = pd.read_csv(input_csv)
sample_df = df.sample(batch_size).reset_index(drop=True)

results = []

# Loop
for idx, msa_sentence in enumerate(tqdm(sample_df['source'], total=len(sample_df))):
    try:
        input_box.clear()
        input_box.send_keys(msa_sentence)
        translate_button.click()

        WebDriverWait(driver, 8).until(
            lambda d: (val := output_box.get_attribute("value").strip()) != "" and val != "Translating..."
        )
        translated_text = output_box.get_attribute("value").strip()

    except Exception as e:
        print(f"❌ Error on sentence {idx}: {msa_sentence}\n{e}")
        translated_text = "ERROR"

    results.append((idx, translated_text))

    # Save every N results
    if (idx + 1) % save_every == 0 or (idx + 1) == len(sample_df):
        print(f"📁 Saving progress at {idx + 1} translations...")
        save_df = sample_df.iloc[[i for i, _ in results]].copy()
        save_df['dialect_translation'] = [t for _, t in results]
        save_df.to_csv(output_csv, mode='a', index=False, header=not os.path.exists(output_csv))
        results = []  # Clear buffer

    time.sleep(1)

driver.quit()
print(f"✅ Done. Results saved to:\n{output_csv}")

SessionNotCreatedException: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir
Stacktrace:
#0 0x567b16a6a29a <unknown>
#1 0x567b164d6040 <unknown>
#2 0x567b16510c69 <unknown>
#3 0x567b1650ba4f <unknown>
#4 0x567b1655c126 <unknown>
#5 0x567b1655b7e6 <unknown>
#6 0x567b1654d463 <unknown>
#7 0x567b16519c55 <unknown>
#8 0x567b1651a8c1 <unknown>
#9 0x567b16a2f1bb <unknown>
#10 0x567b16a330c9 <unknown>
#11 0x567b16a17a39 <unknown>
#12 0x567b16a33c78 <unknown>
#13 0x567b169fc6bf <unknown>
#14 0x567b16a57948 <unknown>
#15 0x567b16a57b26 <unknown>
#16 0x567b16a69176 <unknown>
#17 0x0e0755e1fac3 <unknown>


# Loading Model

### SILMA-2B-4bit

In [ ]:
# !pip install bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
model_id = "silma-ai/SILMA-Kashif-2B-Instruct-v1.0-bnb-4bit" # Quantized version
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config = quantization_config,
)

In [ ]:
messages = [
    {"role": "user", "content": "كم عدد أيام الأسبوع؟"},
]

input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

input_ids

In [ ]:
outputs = model.generate(**input_ids, max_new_tokens=256)

In [ ]:
tokenizer.decode(outputs[0]).split("<start_of_turn>")[-1]

### SILMA-2B

In [ ]:
import torch
from transformers import pipeline

In [ ]:
pipe = pipeline(
    "text-generation",
    model="silma-ai/SILMA-Kashif-2B-Instruct-v1.0",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda"
)

In [ ]:
messages = [
    {"role": "user",
     "content": """ Mask the PII as [masked] in the given source sentence

     source: I sent my email address M.helmy@nu.edu.eg to my supervisor dr. gahda khoriba in Nile University.

     target:
     """
     }
]

outputs = pipe(messages, max_new_tokens=600)
response = outputs[0]["generated_text"][-1]["content"].strip()

In [ ]:
response

In [ ]:
messages = [
    {"role": "user",
     "content": f""" Mask the PII as [masked] in the given source sentence

     source: {df["source"][0]}

     target:
     """
     }
]

outputs = pipe(messages, max_new_tokens=600)
response = outputs[0]["generated_text"][-1]["content"].strip()

In [ ]:
print(df["source"][0])
print(response)